In [1]:
import pandas as pd
import numpy as np
import itertools
import catboost as cat

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score, plot_confusion_matrix, f1_score, accuracy_score, matthews_corrcoef
from catboost import cv, Pool
from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [10]:
def catboost_tune_class(param_comb, X, y, num_iterations, colsample, class_imbal, learning_rate, boosting_type, categorical_feat, cv_metric):

    param_comb = np.hstack((param_comb, np.zeros((param_comb.shape[0], 2))))
    
    #Creating dataset
    ind_cat = X.dtypes == object
    cat_feat = X.columns[ind_cat].tolist()
    cat_dat = Pool(X, y, cat_features=cat_feat)

    #Checking if class imbalance
    if(class_imbal == True): 
        weight_pos = sum(y == 0)/sum(y == 1)
    else: 
        weight_pos = 1

    for i in range(param_comb.shape[0]):
  
        cats_params = {
            'iterations' : num_iterations,
            'learning_rate' : learning_rate,
            'depth' : int(param_comb[i, 0]),
            'rsm' : colsample,
            'loss_function' : 'Logloss',
            'leaf_estimation_iterations' : 1, 
            'leaf_estimation_method' : 'Newton', 
            'random_seed' : 63,
            'verbose' : 10,
            'metric_period' : 1,
            'bagging_temperature' : 1,
            'boosting_type' : boosting_type,
            'simple_ctr' : 'BinarizedTargetMeanValue',
            'bootstrap_type' : 'Bayesian',
            'sampling_frequency' : 'PerTree',
            'scale_pos_weight' : weight_pos,
            'grow_policy' : 'SymmetricTree',
            'score_function' : 'Cosine',
            'custom_loss' : cv_metric, 
            'thread_count' : 1,
            }
        
        cat_cv = cat.cv(params = cats_params,
                        dtrain = cat_dat,
                        nfold = 5,
                        stratified = True,
                        partition_random_seed = 543,
                        early_stopping_rounds = 200,
                        verbose = False)
        
        if(cv_metric == 'AUC'):
            num_tree = np.argmax(cat_cv.loc[:,'test-AUC-mean']) + 1 #number of trees
            opt_val = max(cat_cv.loc[:,'test-AUC-mean'])
        else:
            num_tree = np.argmin(cat_cv.loc[:,'test-Logloss-mean']) + 1 #number of trees
            opt_val = min(cat_cv.loc[:,'test-Logloss-mean'])

        param_comb[i,1:3] = num_tree, opt_val
        
        print(i)
    
    return(param_comb)

In [11]:
def catboost_fit_class(param_opt, X, y, colsample, class_imbal, learning_rate, boosting_type, categorical_feat):

    #Checking if class imbalance
    if(class_imbal == True): 
        weight_pos = sum(y==0)/sum(y==1)
    else: 
        weight_pos = 1
    
    clf_cat = cat.CatBoostClassifier(iterations = int(param_opt[1]),
                                     learning_rate = learning_rate,
                                     depth = int(param_opt[0]),
                                     rsm = colsample,
                                     loss_function = 'Logloss',
                                     leaf_estimation_iterations = 1, 
                                     leaf_estimation_method = 'Newton', 
                                     random_seed = 63,
                                     verbose = False,
                                     metric_period = 1,
                                     bagging_temperature = 1,
                                     boosting_type = boosting_type,
                                     simple_ctr = 'BinarizedTargetMeanValue',
                                     bootstrap_type = 'Bayesian',
                                     sampling_frequency = 'PerTree',
                                     scale_pos_weight = weight_pos,
                                     grow_policy = 'SymmetricTree',
                                     score_function = 'Cosine')

    ind_cat = X.dtypes == object
    cat_feat = X.columns[ind_cat].tolist()
    clf_cat.fit(X, y, cat_features=cat_feat)
    
    return(clf_cat)

In [12]:
#Calculating errors of models
def errors_model(mod, X_test, y_test):
    
    pred_class = mod.predict(X_test) #Class predictions
    acc = accuracy_score(y_test, pred_class) #Accuracy
    f1 = f1_score(y_test, pred_class) #F1 score 
    matt = matthews_corrcoef(y_test, pred_class) #Matthews Correlation Coefficient
    pred_prob = mod.predict_proba(X_test)[:,1] #Probability predictions
    auc = roc_auc_score(y_test, pred_prob) #AUC
    errs_mod = [acc, f1, matt, auc]
    return(errs_mod)

In [5]:
params_depth = np.array(range(1, 11)).reshape(-1,1)

In [13]:
params_depth = np.array(range(1, 2)).reshape(-1,1)

# AdaBoost Overlap

In [6]:
#loading dataset and setting up data. 
import pickle
file_name = 'Overlap_data.pickle'
file = open(file_name,'rb')
Overlap_data = pickle.load(file)
file.close()

X_train_over = Overlap_data['X_train']
y_train_over = Overlap_data['y_train']
X_test_over = Overlap_data['X_test']
y_test_over = Overlap_data['y_test']

In [14]:
#Ordered vs Plain mode of boosting.

#Tuning parameter values for boosting mode = Plain.
##########################################################################################################################
cv_plain_over = catboost_tune_class(param_comb = params_depth, 
                                    X = pd.DataFrame(X_train_over), 
                                    y = y_train_over, 
                                    num_iterations = 5000, 
                                    colsample = 1, 
                                    class_imbal = False, 
                                    learning_rate = 0.05, 
                                    boosting_type = 'Plain',
                                    categorical_feat = False,
                                    cv_metric = 'Logloss')
##########################################################################################################################

#Fitting optimal model for boosting mode = Plain and calculating errors of model.
##########################################################################################################################
opt_ind_plain = np.argmin(cv_plain_over[:,2])

catmod_plain_over = catboost_fit_class(param_opt = cv_plain_over[opt_ind_plain,0:5], 
                                     X = pd.DataFrame(X_train_over), 
                                     y = y_train_over,  
                                     colsample = 1, 
                                     class_imbal = False, 
                                     learning_rate = 0.05, 
                                     boosting_type = 'Plain', 
                                     categorical_feat = False)

errs_plain_over = errors_model(mod = catmod_plain_over, 
                             X_test = X_test_over, 
                             y_test = y_test_over)
##########################################################################################################################


#Tuning parameter values for boosting mode = Ordered.
##########################################################################################################################
cv_ordered_over = catboost_tune_class(param_comb = params_depth, 
                                      X = pd.DataFrame(X_train_over), 
                                      y = y_train_over, 
                                      num_iterations = 5000, 
                                      colsample = 1, 
                                      class_imbal = False, 
                                      learning_rate = 0.05, 
                                      boosting_type = 'Ordered', 
                                      categorical_feat = False,
                                      cv_metric = 'Logloss')
##########################################################################################################################


#Fitting optimal model for boosting mode = Ordered and calculating errors of model.
##########################################################################################################################
opt_ind_ordered = np.argmin(cv_ordered_over[:,2])

catmod_ordered_over = catboost_fit_class(param_opt = cv_ordered_over[opt_ind_ordered,0:5], 
                                         X = pd.DataFrame(X_train_over), 
                                         y = y_train_over, 
                                         colsample = 1, 
                                         class_imbal = False, 
                                         learning_rate = 0.05, 
                                         boosting_type = 'Ordered', 
                                         categorical_feat = False)

#Calculating errors of model
errs_ordered_over = errors_model(mod = catmod_ordered_over, 
                                 X_test = X_test_over, 
                                 y_test = y_test_over)
##########################################################################################################################

Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.


Training on fold [0/5]

bestTest = 0.5339605159
bestIteration = 2501

Training on fold [1/5]

bestTest = 0.5101141235
bestIteration = 3573

Training on fold [2/5]

bestTest = 0.5082684733
bestIteration = 3002

Training on fold [3/5]

bestTest = 0.5036473636
bestIteration = 2767

Training on fold [4/5]

bestTest = 0.524127989
bestIteration = 2283

0


In [21]:
#Table of parameters and test errors for Ordered vs Plain mode of boosting.
test_err_plain = np.append(cv_plain_over[opt_ind_plain,[0,1]],errs_plain_over)
test_err_ordered = np.append(cv_ordered_over[opt_ind_ordered,[0,1]],errs_ordered_over)
test_err_over = np.row_stack((test_err_plain,test_err_ordered))
np.round(test_err_over,2)

array([[   1.  , 3574.  ,    0.77,    0.78,    0.54,    0.84],
       [   1.  , 3829.  ,    0.76,    0.77,    0.52,    0.83]])

# Phoneme

In [22]:
params_depth = np.array([10]).reshape(-1,1)

In [23]:
#Importing and setting up data
file_path_ph = 'C:\\Users\\Matt\\Documents\\Python code thesis\\Datasets\\phoneme.csv'
df_ph = pd.read_csv(file_path_ph)
X_ph = df_ph.drop('target',axis=1).copy()
y_ph = df_ph['target'].copy()
X_train_ph, X_test_ph, y_train_ph, y_test_ph = train_test_split(X_ph, y_ph, random_state=65, stratify=y_ph, test_size=0.2)

In [24]:
#Tuning parameter values for boosting mode = Plain 
cv_plain_ph = catboost_tune_class(param_comb = params_depth, 
                                  X = X_train_ph, 
                                  y = y_train_ph, 
                                  num_iterations = 5000, 
                                  colsample = 1, 
                                  class_imbal = True, 
                                  learning_rate = 0.05, 
                                  boosting_type = 'Plain', 
                                  categorical_feat = False,
                                  cv_metric = 'AUC')

Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.


Training on fold [0/5]

bestTest = 0.2244846391
bestIteration = 1807

Training on fold [1/5]

bestTest = 0.2665431216
bestIteration = 1385

Training on fold [2/5]

bestTest = 0.2712592201
bestIteration = 1263

Training on fold [3/5]

bestTest = 0.3229107711
bestIteration = 753

Training on fold [4/5]

bestTest = 0.273125865
bestIteration = 1120

0


In [25]:
#Fitting optimal model for boosting mode = Plain 
opt_ind_plain = np.argmax(cv_plain_ph[:,2])

catmod_plain_ph = catboost_fit_class(param_opt = cv_plain_ph[opt_ind_plain,0:5], 
                                     X = X_train_ph, 
                                     y = y_train_ph,  
                                     colsample = 1, 
                                     class_imbal = True, 
                                     learning_rate = 0.05, 
                                     boosting_type = 'Plain', 
                                     categorical_feat = False)

#Calculating errors of model
errs_plain_ph = errors_model(mod = catmod_plain_ph, 
                             X_test = X_test_ph, 
                             y_test = y_test_ph)

Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.


In [26]:
#Tuning parameter values for boosting mode = Ordered
cv_ordered_ph = catboost_tune_class(param_comb = params_depth, 
                                    X = X_train_ph, 
                                    y = y_train_ph, 
                                    num_iterations = 5000, 
                                    colsample = 1, 
                                    class_imbal = True, 
                                    learning_rate = 0.05, 
                                    boosting_type = 'Ordered', 
                                    categorical_feat = False,
                                    cv_metric = 'AUC')

Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.


Training on fold [0/5]


KeyboardInterrupt: 

In [ ]:
#Fitting optimal model for boosting mode = Ordered
opt_ind_ordered = np.argmax(cv_ordered_ph[:,2])

catmod_ordered_ph = catboost_fit_class(param_opt = cv_ordered_ph[opt_ind_ordered,0:5], 
                                       X = X_train_ph, 
                                       y = y_train_ph, 
                                       colsample = 1, 
                                       class_imbal = True, 
                                       learning_rate = 0.1, 
                                       boosting_type = 'Ordered', 
                                       categorical_feat = False)

#Calculating errors of model
errs_ordered_ph = errors_model(mod = catmod_ordered_ph, 
                               X_test = X_test_ph, 
                               y_test = y_test_ph)

In [ ]:
#Table of parameters and test errors 
test_err_plain = np.append(cv_plain_ph[opt_ind_plain,[0,1]],errs_plain_ph)
test_err_ordered = np.append(cv_ordered_ph[opt_ind_ordered,[0,1]],errs_ordered_ph)
test_err_ph = np.row_stack((test_err_plain,test_err_ordered))
np.round(test_err_ph,3)

# Adult

In [34]:
params_depth = np.array([8]).reshape(-1,1)

In [35]:
file_path = 'C:\\Users\\Matt\\Documents\\Python code thesis\\Datasets\\adult1.csv'
df_adult1 = pd.read_csv(file_path,header=None)
file_path = 'C:\\Users\\Matt\\Documents\\Python code thesis\\Datasets\\adult2.csv'
df_adult2 = pd.read_csv(file_path,header=None)
df_adult = pd.concat([df_adult1, df_adult2])
df_adult.columns = ["age","workclass","fnlwgt","education","education-num","martial-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country","target"]
df_adult.replace(' ','',regex=True,inplace=True)
df_adult['target'] = df_adult['target'].apply(lambda x: x.rstrip('.'))
df_adult['target'] = df_adult['target'].apply(lambda x: 0 if x=="<=50K" else 1)

In [36]:
X_adult = df_adult.drop('target', axis = 1).copy()
y_adult = df_adult['target'].copy()
X_train_adult, X_test_adult, y_train_adult, y_test_adult = train_test_split(X_adult, y_adult, random_state=65, stratify=y_adult, test_size=0.2)

In [38]:
#Tuning parameters for boosting mode = Plain 
cv_plain_adult = catboost_tune_class(param_comb = params_depth, 
                                  X = X_train_adult, 
                                  y = y_train_adult,
                                  num_iterations = 1000, 
                                  colsample = 1, 
                                  class_imbal = True, 
                                  learning_rate = 0.05, 
                                  boosting_type = 'Plain', 
                                  categorical_feat = True,
                                  cv_metric = 'AUC')

Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.


Training on fold [0/5]

bestTest = 0.3395417706
bestIteration = 952

Training on fold [1/5]

bestTest = 0.3493895368
bestIteration = 783

Training on fold [2/5]

bestTest = 0.3391214828
bestIteration = 659

Training on fold [3/5]

bestTest = 0.3392011046
bestIteration = 796

Training on fold [4/5]

bestTest = 0.3474974762
bestIteration = 607

0


In [39]:
#Fitting optimal model for boosting mode = Plain 
opt_ind_plain = np.argmax(cv_plain_adult[:,2])

catmod_plain_adult = catboost_fit_class(param_opt = cv_plain_adult[opt_ind_plain,:], 
                                     X = X_train_adult, 
                                     y = y_train_adult, 
                                     colsample = 1, 
                                     class_imbal = True, 
                                     learning_rate = 0.05, 
                                     boosting_type = 'Plain', 
                                     categorical_feat = True)

#Calculating errors of model
errs_plain_adult = errors_model(mod = catmod_plain_adult, 
                                X_test = X_test_adult, 
                                y_test = y_test_adult)

Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.


In [40]:
#Tuning parameter values for boosting mode = Ordered
cv_ordered_adult = catboost_tune_class(param_comb = params_depth, 
                                    X = X_train_adult, 
                                    y = y_train_adult, 
                                    num_iterations = 1000, 
                                    colsample = 1, 
                                    class_imbal = True, 
                                    learning_rate = 0.05, 
                                    boosting_type = 'Ordered', 
                                    categorical_feat = True,
                                    cv_metric = 'AUC')

Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.


Training on fold [0/5]

bestTest = 0.3310437155
bestIteration = 982

Training on fold [1/5]

bestTest = 0.338511412
bestIteration = 952

Training on fold [2/5]

bestTest = 0.3316027927
bestIteration = 869

Training on fold [3/5]

bestTest = 0.3303092149
bestIteration = 996

Training on fold [4/5]

bestTest = 0.3357620511
bestIteration = 879

0


In [41]:
#Fitting optimal model for boosting mode = Ordered
opt_ind_ordered = np.argmax(cv_ordered_adult[:,2])

catmod_ordered_adult = catboost_fit_class(param_opt = cv_ordered_adult[opt_ind_ordered,0:5], 
                                       X = X_train_adult, 
                                       y = y_train_adult, 
                                       colsample = 1, 
                                       class_imbal = True, 
                                       learning_rate = 0.05, 
                                       boosting_type = 'Ordered', 
                                       categorical_feat = True)

#Calculating errors of model
errs_ordered_adult = errors_model(mod = catmod_ordered_adult, 
                             X_test = X_test_adult, 
                             y_test = y_test_adult)

Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.


In [42]:
#Table of parameters and test errors 
test_err_plain = np.append(cv_plain_adult[opt_ind_plain,[0,1]],errs_plain_adult)
test_err_ordered = np.append(cv_ordered_adult[opt_ind_ordered,[0,1]],errs_ordered_adult)
test_err_adult = np.row_stack((test_err_plain,test_err_ordered))
np.round(test_err_adult,3)

array([[   8.   ,  795.   ,    0.837,    0.72 ,    0.629,    0.933],
       [   8.   , 1000.   ,    0.837,    0.72 ,    0.629,    0.933]])

In [76]:
cv_ordered_adult[opt_ind_ordered,:]

array([   8.        , 1000.        ,    0.92928774])

# Santander Customer Satisfaction

In [87]:
#loading dataset
import pickle
file_name = 'customersatformat.pickle'
file = open(file_name,'rb')
df_sat = pickle.load(file)
file.close()

In [88]:
X_sat = df_sat.drop('TARGET', axis = 1).copy()
y_sat = df_sat['TARGET'].copy()
X_train_sat, X_test_sat, y_train_sat, y_test_sat = train_test_split(X_sat, y_sat, random_state=71, stratify=y_sat, test_size=0.2)

In [16]:
#Tuning parameters for boosting mode = Plain 
cv_plain_sat = catboost_tune_class(param_comb = params_depth, 
                                  X = X_train_sat, 
                                  y = y_train_sat, 
                                  num_iterations = 5000, 
                                  colsample = 0.3, 
                                  class_imbal = True, 
                                  learning_rate = 0.05, 
                                  boosting_type = 'Plain', 
                                  categorical_feat = False,
                                  metric='AUC')

Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.


Training on fold [0/5]

bestTest = 0.4915345675
bestIteration = 99

Training on fold [1/5]

bestTest = 0.5234356757
bestIteration = 98

Training on fold [2/5]

bestTest = 0.4995555901
bestIteration = 99

Training on fold [3/5]

bestTest = 0.5095987255
bestIteration = 99

Training on fold [4/5]

bestTest = 0.5152597185
bestIteration = 99

0
Training on fold [0/5]


Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.



bestTest = 0.4831279368
bestIteration = 99

Training on fold [1/5]

bestTest = 0.5196399421
bestIteration = 98

Training on fold [2/5]

bestTest = 0.4906952434
bestIteration = 99

Training on fold [3/5]

bestTest = 0.505272524
bestIteration = 98

Training on fold [4/5]

bestTest = 0.5114450284
bestIteration = 97

1


In [17]:
#Fitting optimal model for boosting mode = Plain 
opt_ind_plain = np.argmin(cv_plain_sat[:,2])

catmod_plain_sat = catboost_fit_class(param_opt = cv_plain_sat[opt_ind_plain,:], 
                                     X = X_train_sat, 
                                     y = y_train_sat,  
                                     colsample = 0.3, 
                                     class_imbal = True, 
                                     learning_rate = 0.05, 
                                     boosting_type = 'Plain', 
                                     categorical_feat = True)

#Calculating errors of model
errs_plain_sat = errors_model(mod = catmod_plain_sat, 
                             X_test = X_test_sat, 
                             y_test = y_test_sat)

Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.


0:	learn: 0.6798010	total: 351ms	remaining: 34.8s
10:	learn: 0.5906817	total: 3.85s	remaining: 31.1s
20:	learn: 0.5489978	total: 7.25s	remaining: 27.3s
30:	learn: 0.5277998	total: 10.7s	remaining: 23.8s
40:	learn: 0.5154511	total: 14.1s	remaining: 20.3s
50:	learn: 0.5076636	total: 17.7s	remaining: 17s
60:	learn: 0.5015902	total: 21.4s	remaining: 13.7s
70:	learn: 0.4979204	total: 24.9s	remaining: 10.2s
80:	learn: 0.4950663	total: 28.4s	remaining: 6.66s
90:	learn: 0.4923824	total: 31.9s	remaining: 3.16s
99:	learn: 0.4903611	total: 35s	remaining: 0us


In [19]:
#Tuning parameter values for boosting mode = Ordered
cv_ordered_sat = catboost_tune_class(param_comb = params_depth, 
                                    X = X_train_sat, 
                                    y = y_train_sat, 
                                    num_iterations = 5000, 
                                    colsample = 0.3, 
                                    class_imbal = True, 
                                    learning_rate = 0.05, 
                                    boosting_type = 'Ordered', 
                                    categorical_feat = True)

Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.


Training on fold [0/5]

bestTest = 0.4928853309
bestIteration = 99

Training on fold [1/5]

bestTest = 0.5254160024
bestIteration = 99

Training on fold [2/5]

bestTest = 0.5006916553
bestIteration = 99

Training on fold [3/5]

bestTest = 0.5102025688
bestIteration = 99

Training on fold [4/5]

bestTest = 0.5167438337
bestIteration = 99

0
Training on fold [0/5]


Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.



bestTest = 0.4830293898
bestIteration = 99

Training on fold [1/5]

bestTest = 0.5219971892
bestIteration = 96

Training on fold [2/5]

bestTest = 0.4931625813
bestIteration = 99

Training on fold [3/5]

bestTest = 0.5066379311
bestIteration = 99

Training on fold [4/5]

bestTest = 0.5118442145
bestIteration = 99

1


In [20]:
#Fitting optimal model for boosting mode = Ordered
opt_ind_ordered = np.argmin(cv_ordered_sat[:,2])

catmod_ordered_sat = catboost_fit_class(param_opt = cv_ordered_sat[opt_ind_ordered,:], 
                                       X = X_train_sat, 
                                       y = y_train_sat,  
                                       colsample = 0.3, 
                                       class_imbal = True, 
                                       learning_rate = 0.05, 
                                       boosting_type = 'Ordered', 
                                       categorical_feat = True)

#Calculating errors of model
errs_ordered_sat = errors_model(mod = catmod_ordered_sat, 
                             X_test = X_test_sat, 
                             y_test = y_test_sat)

Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.


0:	learn: 0.6798010	total: 400ms	remaining: 39.6s
10:	learn: 0.5906817	total: 4.4s	remaining: 35.6s
20:	learn: 0.5489978	total: 8.36s	remaining: 31.5s
30:	learn: 0.5277998	total: 12.5s	remaining: 27.7s
40:	learn: 0.5154511	total: 16.5s	remaining: 23.8s
50:	learn: 0.5076636	total: 20.7s	remaining: 19.9s
60:	learn: 0.5015902	total: 24.8s	remaining: 15.9s
70:	learn: 0.4979204	total: 28.8s	remaining: 11.8s
80:	learn: 0.4950663	total: 32.9s	remaining: 7.73s
90:	learn: 0.4923824	total: 37s	remaining: 3.66s
99:	learn: 0.4903611	total: 40.6s	remaining: 0us


In [21]:
#Table of parameters and test errors 
test_err_plain = np.append(cv_plain_sat[opt_ind_plain,0],errs_plain_sat)
test_err_ordered = np.append(cv_ordered_sat[opt_ind_ordered,0],errs_ordered_sat)
test_err_sat = np.row_stack((test_err_plain,test_err_ordered))
np.round(test_err_sat,3)

array([[8.   , 0.766, 0.2  , 0.227, 0.831],
       [8.   , 0.766, 0.2  , 0.227, 0.831]])

In [19]:
np.set_printoptions(suppress=True)